In [1]:
%matplotlib inline
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns; sns.set()

In [2]:
import lightgbm as lgb
from sklearn.model_selection import KFold, StratifiedKFold
from sklearn.metrics import roc_auc_score, roc_curve

In [3]:
train = pd.read_csv('./data/train.csv')
test = pd.read_csv('./data/test.csv')

In [6]:
x = train.iloc[:, 2:]

In [8]:
for col in x.columns:
    x[col + '_square'] = x[col] ** 2

In [12]:
target = train.target

In [15]:
train = x

In [16]:
# https://www.kaggle.com/fayzur/customer-transaction-prediction-strong-baseline
# Thanks fayzur. Nice Parameter 
param = {
        'num_leaves': 10,
        'max_bin': 119,
        'min_data_in_leaf': 11,
        'learning_rate': 0.02,
        'min_sum_hessian_in_leaf': 0.00245,
        'bagging_fraction': 1.0, 
        'bagging_freq': 5, 
        'feature_fraction': 0.05,
        'lambda_l1': 4.972,
        'lambda_l2': 2.276,
        'min_gain_to_split': 0.65,
        'max_depth': -1,
        'save_binary': True,
        'seed': 1337,
        'feature_fraction_seed': 1337,
        'bagging_seed': 1337,
        'drop_seed': 1337,
        'data_random_seed': 1337,
        'objective': 'binary',
        'boosting_type': 'gbdt',
        'verbose': 1,
        'metric': 'auc',
        'is_unbalance': True,
        'boost_from_average': False,
    }

In [32]:
folds = StratifiedKFold(n_splits=5, shuffle=True, random_state=42)

oof_lgb = np.zeros(len(train))
predictions_lgb = np.zeros(len(test))
feature_importance = pd.DataFrame()

train_columns = [c for c in train.columns if c not in ['ID_code', 'target']]
train_coumns = new_feature

for fold_, (trn_idx, val_idx) in enumerate(folds.split(train, target.values)):    
    print("fold n°{}".format(fold_))
    trn_data = lgb.Dataset(train.iloc[trn_idx][train_columns], label=target.iloc[trn_idx])
    val_data = lgb.Dataset(train.iloc[val_idx][train_columns], label=target.iloc[val_idx])

    num_round = 60000
    clf = lgb.train(param, trn_data, num_round, valid_sets = [trn_data, val_data], verbose_eval=3000, early_stopping_rounds = 200)
    oof_lgb[val_idx] = clf.predict(train.iloc[val_idx][train_columns], num_iteration=clf.best_iteration)
    
#     predictions_lgb += clf.predict(test[train_columns], num_iteration=clf.best_iteration) / folds.n_splits
    
    fold_importance = pd.DataFrame()
    fold_importance["Feature"] = train_columns
    fold_importance["importance"] = clf.feature_importance()
    fold_importance["fold"] = fold_ + 1
    feature_importance = pd.concat([feature_importance, fold_importance], axis=0)
    
    print("CV score: {:<8.5f}".format(roc_auc_score(target.values[val_idx], oof_lgb[val_idx])))
    
print("CV score: {:<8.5f}".format(roc_auc_score(target.values, oof_lgb)))

fold n°0
Training until validation scores don't improve for 200 rounds.
[3000]	training's auc: 0.934772	valid_1's auc: 0.895867
Early stopping, best iteration is:
[4197]	training's auc: 0.944433	valid_1's auc: 0.897471
CV score: 0.89747 
fold n°1
Training until validation scores don't improve for 200 rounds.
[3000]	training's auc: 0.934515	valid_1's auc: 0.896634
Early stopping, best iteration is:
[5352]	training's auc: 0.952389	valid_1's auc: 0.898661
CV score: 0.89866 
fold n°2
Training until validation scores don't improve for 200 rounds.
[3000]	training's auc: 0.933475	valid_1's auc: 0.901804
Early stopping, best iteration is:
[5554]	training's auc: 0.953028	valid_1's auc: 0.90409
CV score: 0.90409 
fold n°3
Training until validation scores don't improve for 200 rounds.
[3000]	training's auc: 0.934881	valid_1's auc: 0.894093
Early stopping, best iteration is:
[4850]	training's auc: 0.94922	valid_1's auc: 0.896207
CV score: 0.89621 
fold n°4
Training until validation scores don't im

In [23]:
feature_importance.max()

Feature       var_9_square
importance             327
fold                     5
dtype: object

In [31]:
new_feature = feature_importance.groupby('Feature').importance.mean().sort_values(ascending = False)[:200].index